In [127]:
# Austin Corgan
# ASTR 400B Research Project

# Topic: Fate of solar analogs in M31 and M33

# Question: How do the positions of solar analogs evolve? 

# Plot XY positions at various points in time with histograms of distance
# as well as determine the fraction of stars beginning as solar analogs which 
# become unbound to their host galaxy as a function of time

In [77]:
# import modules

from ReadFile import Read
from CenterOfMass2 import CenterOfMass
import numpy as np
import astropy.units as u
from astropy.constants import G
import matplotlib
import matplotlib.pyplot as plt 

G = G.to(u.kpc*u.km**2/u.s**2/u.Msun)

In [78]:
# Get center of mass of M31, M33 by making instances of CenterOfMass class at Snap 0 
# Subscript "0" will denote quantities related to Snap 0 

COMM31_0 = CenterOfMass('M31_000.txt',2) # "2" indicates use of disk particles
COMM33_0 = CenterOfMass('M33_000.txt',2)

# Get COM positions with COM_P function from CenterOfMass class

M31COMP_0 = COMM31_0.COM_P(0.1,2)
M33COMP_0 = COMM33_0.COM_P(0.1,4)

M31COMV_0 = COMM31_0.COM_V(M31COMP_0[0],M31COMP_0[1],M31COMP_0[2]).value
M33COMV_0 = COMM33_0.COM_V(M33COMP_0[0],M33COMP_0[1],M33COMP_0[2]).value

M31COMP_0 = M31COMP_0.value
M33COMP_0 = M33COMP_0.value

In [79]:
# Create function to determine magnitude of difference between two vectors 
# Here, will be used to find distance of points in galaxies from COM of that galaxy 

def magdiff(ax,ay,az,bx,by,bz):
    return np.sqrt((ax-bx)**2+(ay-by)**2+(az-bz)**2)

In [80]:
# Store data array at t = 0 using Read function, getting initial position and velocity vectors

M31time_0, M31total_0, M31data_0 = Read('M31_000.txt')
M33time_0, M33total_0, M33data_0 = Read('M33_000.txt')


In [81]:
def JacobiRadius(r,Msat,Mhost): 
    # Function to compute the Jacobi Radius for a satellite (M33) of a host (M31)
    # This will be used to compute the mass of M33 at each snap, as the mass will be changing
    # Note that this assumes host galaxy is an isothermal sphere
    # Inputs:
    #    r - distance between the satellite and the host (kpc)
    #    Msat - total mass of satellite in Msun
    #    Mhost - mass of the host enclosed within r in Msun
    # Returns: 
    #    Jacobi Radius in kpc
    
    return r*(Msat/2/Mhost)**(1/3)

In [82]:
def MassEnclosed(filename,rmax,COMP):
    # Function to compute total mass enclosed within a given distance of the center of mass
    # Inputs: 
    #    filename - name of file
    #    rmax - radius to compute mass enclosed within (kpc)
    #    COMP - center of mass position array (x,y,z) of galaxy (kpc)
    # Returns:
    #    total mass enclosed within rmax (Msun)
    
    time, total, data = Read(filename)
    
    # Get distance between each particle and center of mass
    r = magdiff(data['x'],data['y'],data['z'],COMP[0],COMP[1],COMP[2])
    
    # pick out particles which are within desired radius 
    index = np.where(r < rmax)
    
    return data['m'][index].sum()*1e10

In [83]:
def HaloMassEnclosed(filename,rmax,COMP):
    # Function to compute total mass enclosed within a given distance of the center of mass
    # Inputs: 
    #    filename - name of file
    #    rmax - radius to compute mass enclosed within (kpc)
    #    COMP - center of mass position array (x,y,z) of galaxy (kpc)
    # Returns:
    #    halo mass enclosed within rmax (Msun)
    
    time, total, data = Read(filename)
    
    # Get distance between each particle and center of mass
    r = magdiff(data['x'],data['y'],data['z'],COMP[0],COMP[1],COMP[2])
    
    # pick out particles which are within desired radius 
    index = np.where((r < rmax) & (data['type'] == 1))
    
    return data['m'][index].sum()*1e10

In [ ]:
# Will create an array which contains the halo mass of M33 in Msun at each snapnumber
# This will be used later to determine the escape velocity at each snapnumber

# initialize arrays, will need to make one for total mass as well because this is how the Jacobi radius is determined
M33Mass = np.zeros(802)
M33HaloMass = np.zeros(802)

# use total and halo mass at present day as 0th entry to arrays (using value from Homework 3)
M33Mass[0] = 0.196e12
M33HaloMass[0] = 0.187e12

# loop over all snapnumbers 
for i in np.arange(1,802,1):
    
    # create filenames from index
    ilbl = '000' + str(i)
    ilbl = ilbl[-3:]
    M31filename='M31_' + ilbl + '.txt'
    M33filename='M33_' + ilbl + '.txt'
    
    # get center of mass positions of M33 and M31 at each snap
    COMM31 = CenterOfMass(M31filename,2) # "2" indicates use of disk particles
    COMM33 = CenterOfMass(M33filename,2)
    M31COMP = COMM31.COM_P(0.1,2).value
    M33COMP = COMM33.COM_P(0.1,4).value
    
    # determine the separation of the two galaxies 
    r = magdiff(M31COMP[0],M31COMP[1],M31COMP[2],M33COMP[0],M33COMP[1],M33COMP[2])
    
    # get the mass of M31 contained within the separation between M31 and M33
    Mhost = MassEnclosed(M31filename,r,M31COMP)
    
    # use separation between galaxies, mass of M31 just calculated, and previous value of mass of M33 to 
    # get Jacobi radius
    Rj = JacobiRadius(r,M33Mass[i-1],Mhost)
    
    # Now sum the mass of all the particles within the Jacobi radius to get the new mass of M33
    M33Mass[i] = MassEnclosed(M33filename,Rj,M33COMP)
    M33HaloMass[i] = HaloMassEnclosed(M33filename,Rj,M33COMP)


In [84]:
# a function that will rotate the position and velocity vectors
# so that the disk angular momentum is aligned with z axis. 

def RotateFrame(posI,velI):
    # input:  3D array of positions and velocities
    # returns: 3D array of rotated positions and velocities such that j is in z direction

    # compute the angular momentum
    L = np.sum(np.cross(posI,velI), axis=0)
    # normalize the vector
    L_norm = L/np.sqrt(np.sum(L**2))


    # Set up rotation matrix to map L_norm to z unit vector (disk in xy-plane)
    
    # z unit vector
    z_norm = np.array([0, 0, 1])
    
    # cross product between L and z
    vv = np.cross(L_norm, z_norm)
    s = np.sqrt(np.sum(vv**2))
    
    # dot product between L and z 
    c = np.dot(L_norm, z_norm)
    
    # rotation matrix
    I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    v_x = np.array([[0, -vv[2], vv[1]], [vv[2], 0, -vv[0]], [-vv[1], vv[0], 0]])
    R = I + v_x + np.dot(v_x, v_x)*(1 - c)/s**2

    # Rotate coordinate system
    pos = np.dot(R, posI.T).T
    vel = np.dot(R, velI.T).T
    
    return pos, vel

In [101]:
# Make index to pick out solar analogs 
# Using distance of Sun from center of MW as 8.178 kpc (from gravity collaboration - see Lecture 4)
# Can set index to pick out particles within some delta of this radius from COM 


# Determine disk particles which have distance relative to COM within delta of 8.178 kpc 
# Delta is different between M31 and M33
# Larger for M33 to ensure we have ~ 500-1000 particles to work with

M31r = magdiff(M31data_0['x'],M31data_0['y'],M31data_0['z'],M31COMP_0[0],M31COMP_0[1],M31COMP_0[2])

M31index = np.where((M31data_0['type']==2) & (abs(M31r - 8.178) < 0.025))

M33r = magdiff(M33data_0['x'],M33data_0['y'],M33data_0['z'],M33COMP_0[0],M33COMP_0[1],M33COMP_0[2])

M33index = np.where((M33data_0['type']==2) & (abs(M33r - 8.178) < 0.3))



In [111]:
def SolarAnalogGraphs(galaxy,snap):
    # This function will give the positions of all particles which began as solar analogs in a given galaxy 
    # at a given point in time
    # Inputs:
    #    galaxy - name of galaxy to examine; either 'M31' or 'M33'
    #    snap - snap number
    # Returns: 
    #    an x-y plane showing the positions of particles (which started out as solar analogs) at a given snap number 
    
    # construct corresponding file name from input 
    ilbl = '000' + str(snap)
    ilbl = ilbl[-3:]
    filename="%s_"%(galaxy) + ilbl + '.txt'
    
    # read in data from file
    time, total, data = Read(filename)
    
    # making instance of CenterOfMass class corresponding to input
    COM = CenterOfMass(filename,2)
    
    # get COM at current snap number based on galaxy (VolDec is different depending on M31 or M33)
    if (galaxy == 'M31'):
        COMP = COM.COM_P(0.1,2)
        
    if (galaxy == 'M33'):
        COMP = COM.COM_P(0.1,4)
        
    COMV = COM.COM_V(COMP[0],COMP[1],COMP[2]).value
    
    COMP = COMP.value
        
    # get position and velocity of all particles relative to current COM
    x = data['x'] - COMP[0]
    y = data['y'] - COMP[1]
    z = data['z'] - COMP[2]
    vx = data['vx'] - COMV[0]
    vy = data['vy'] - COMV[1]
    vz = data['vz'] - COMV[2]
    
    
    # Make position and velocity arrays to give to RotateFrame function
    PositionArray = np.array([x,y,z]).T
    VelocityArray = np.array([vx,vy,vz]).T
    
    # Rotate frame
    RotatedPositionArray, RotatedVelocityArray = RotateFrame(PositionArray,VelocityArray)
    
    # Pick out particles which started as solar analogs 
    
    if (galaxy == 'M31'):
        SolarAnalogPositions = RotatedPositionArray[M31index]
        SolarAnalogVelocities = RotatedVelocityArray[M31index]
    
    if (galaxy == 'M33'):
        SolarAnalogPositions = RotatedPositionArray[M33index]
        SolarAnalogVelocities = RotatedVelocityArray[M33index]
        
    # plot x-y plane, rotated such that L along z-axis
    
    fig = plt.figure(figsize=(3.5,3.5))
    ax = plt.subplot(111)

    # Plot position of particles relative to COM
    plt.scatter(SolarAnalogPositions[:,0],SolarAnalogPositions[:,1],color='blue')

    # Add chart title 
    plt.title(str(galaxy) + ' at t = '+ str(np.round(time.to(u.Gyr),2)),fontsize=15)

    # Add axis labels
    plt.xlabel('x (kpc)',fontsize=12)
    plt.ylabel('y (kpc)',fontsize=12)

    # set axis limits
    plt.xlim(-100,100)
    plt.ylim(-100,100)


    # add a legend
    # legend = ax.legend(loc='upper right',fontsize='x-large')
    
    
    # Get magnitude of position vector 
    r = np.sqrt((SolarAnalogPositions[:,0]**2 + SolarAnalogPositions[:,1]**2 + SolarAnalogPositions[:,2]**2))
    
    
    # plot histogram of radii 
    fig, ax=plt.subplots(figsize=(3.5,3.5))
    
    plt.hist(r,bins=np.arange(0,max(100,max(r)+5),5),ec='black')
    
    plt.xlim(0,max(100,max(r)+5))
    
    plt.xlabel('r (kpc)',fontsize=12)
    plt.ylabel('Number',fontsize=12)

In [41]:
def HernquistPotential(r,a,M):
    # Function to compute the 1990 Hernquist potential function (See lecture 4)
    # This will help in determining if a particle becomes unbound 
    # Hernquist potential is phi = -GM/(r+a)
    # Inputs: 
    #    M - total dark matter halo mass Msun
    #    a - scale radius in kpc
    #    r - distance from center of galaxy in kpc
    # Returns: 
    #    value of Hernquist potential at a given radius in km^2/s^2
    
    return -G*M/(r+a)
    
    

In [42]:
def PercentUnbound(galaxy,snap):
    # This function will determine the percentage of particles which started out as solar analogs which 
    # have become unbound at the given snap number. Particles will be considered unbound if they have 
    # a velocity greater than the escape velocity as determined by the Hernquist Profile phi. i.e.,
    # if v > sqrt(2*abs(phi))
    # Inputs: 
    #    galaxy - name of galaxy to examine; either 'M33' or 'M31'
    #    snap - snap number
    # Returns: 
    #    graph of percentage of solar analogs which are unbound as a function of time
    
    # construct corresponding file name from input 
    ilbl = '000' + str(snap)
    ilbl = ilbl[-3:]
    filename="%s_"%(galaxy) + ilbl + '.txt'
    
    # read in data from file
    time, total, data = Read(filename)
    
    time = time.value/1000. # convert time to Gyr (Read outputs time in Myr)
    
    # making instance of CenterOfMass class corresponding to input
    COM = CenterOfMass(filename,2)
    
    # get COM position at current snap number based on galaxy (VolDec is different depending on M31 or M33)
    if (galaxy == 'M31'):
        COMP = COM.COM_P(0.1,2)
        
    if (galaxy == 'M33'):
        COMP = COM.COM_P(0.1,4)
        
    # get COM velocity using COM position 
    
    COMV = COM.COM_V(COMP[0],COMP[1],COMP[2]).value
    
    COMP = COMP.value
        
    # get position of particles which started out as solar analogs relative to current COM
    if (galaxy == 'M31'):
        x = data['x'][M31index] - COMP[0]
        y = data['y'][M31index] - COMP[1]
        z = data['z'][M31index] - COMP[2]
        
    if (galaxy == 'M33'):
        x = data['x'][M33index] - COMP[0]
        y = data['y'][M33index] - COMP[1]
        z = data['z'][M33index] - COMP[2]
        
    # get distance between particle and COM
    r = np.sqrt(x**2+y**2+z**2)
    
    # get velocities of particles which started out as solar analogs relative to current COM
    if (galaxy == 'M31'):
        vx = data['vx'][M31index] - COMV[0]
        vy = data['vy'][M31index] - COMV[1]
        vz = data['vz'][M31index] - COMV[2]
        
    if (galaxy == 'M33'):
        vx = data['vx'][M33index] - COMV[0]
        vy = data['vy'][M33index] - COMV[1]
        vz = data['vz'][M33index] - COMV[2]
        
    # get magnitude of relative velocity between particle and COM
    v = np.sqrt(vx**2+vy**2+vz**2)
        
    # determine v_esc, will vary depending on galaxy, as v_esc depends on Hernquist potential, which depends on 
    # total dark matter halo mass and scale radius. Will use total halo masses for M31 (1.975e12 Msun) as determined
    # in Homework 3. Mass of M33 changes with time, so will call the corresponding element of the previously 
    # calculated array
    # Will use scale radii for M31 (62 kpc) and M33 (25 kpc) as determined in Homework 5. Scale radius for M33 should
    # also be changing with time, but this is not taken into account
    
    if (galaxy == 'M31'): 
        vesc = np.sqrt(2*abs(HernquistPotential(r,62,1.975e12))).value
        
    if (galaxy == 'M33'):
        vesc = np.sqrt(2*abs(HernquistPotential(r,25,M33HaloMass[snap]))).value
        
    # Now count how many particles have velocity relative to the COM of the galaxy which is greater than v_esc
    
    n = 0 # initialize an index, will represent number of particles with v > vesc
    
    for i in np.arange(0,np.size(v)):
        if v[i] > vesc[i]:
            n = n+1
    
    # Divide n by total number of particles to return fraction which are unbound 
    return n/np.size(v), time
    
        
    

In [74]:
def FractionGraph(galaxy):
    # This function will give the fraction of unbounded particles as a function of time, where the fraction
    # unbound at any given time will be computed by the above PercentUnbound function
    # Inputs: 
    #    galaxy - either 'M31' or 'M33'
    #    interval - positive integer indicating time step in snap numbers 
    # Returns:
    #    A graph of fraction of particles unbound as a function of time
    
    # Initialize time and fraction arrays, which will end up being the (x,y) coordinates of the graph
    
    time = np.zeros(802)
    fraction = np.zeros(802)
    
    # loop over every snap, calling the above PercentUnbound function each time
    
    for i in np.arange(0,802,1): 
        fraction[i],time[i] = PercentUnbound(galaxy,i)
        
    # graph fraction unbound vs. time
    
    fig = plt.figure(figsize=(10,10))
    ax = plt.subplot(111)

    plt.plot(time,fraction,color='blue')


    # Add axis labels
    plt.xlabel('Time (Gyr)',fontsize=12)
    plt.ylabel('Fraction Unbound',fontsize=12)

    # set axis limits
    plt.xlim(0,12)
    plt.ylim(0,1)
    
    return 